
## Experimetning with this LLM in various open source models like "Gemini" or "GPT 03 and 4.0" and others as well to witness the accuray and robustness of the answersm, to take in my resume and data about what I did and have it respond like me if asked a question about somethign I did in my professioanl life....

In [1]:
# If you don't have any of the modules installed, you have to install them first...
!pip install pypdf python-dotenv openai gradio

In [4]:
### And make sure your kernel displayes so you can select the proper one....
!python -m ipykernel install --user --name=myenv --display-name="Python (myenv)"

Installed kernelspec myenv in C:\Users\chris' pc\AppData\Roaming\jupyter\kernels\myenv


In [4]:
load_dotenv(override=True)
openai = OpenAI()

In [5]:
reader = PdfReader("llm_research\me\ChristopherMR-resume.pdf")
linkedin = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

        

NameError: name 'PdfReader' is not defined

In [ ]:
print(linkedin)


In [ ]:
## Read in the specific data for the LLM to get answers from....

with open("llm_research\me\ChristopherMR-resume.pdf", "r", encoding="utf-8") as f:
    summary = f.read()

In [ ]:
name = "Chris Morris"

In [ ]:
## Prompt engineering sections...

system_prompt = f"I want you to act like me or {name}. You are answering questions on my behalf and from \
    {name} file, resume, and website. Particulary questions related to {name}'s career, bacground, skills, experience \
        so if you have a job interview there will be a job decription provided and you will use {name}'s resume, skills, website \
            and otehr documents to answer the most accurate and make {name} the most qualified BUT NOT sounding \
        over-qualifed like he's a robot. Mkae the answers sound natural and human like {name} talks. \
        Also, be professional, and engaging, as if talking to potential employer or boss."

system_prompt += f"\n\n## Resume: \n{ChristopherMR-resume}\n\n ## some things I did at Mercola Health: \n{Mobile App & EMR}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."



In [ ]:
system_prompt


In [ ]:
## Create a function here to take in the messages and return the responses....

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].messages.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

### Using a LLM "Gemini" to act as a "evaluator" LLM to evaluate or check to reponse of the output 
of the other LLM agent with NO agenitc framework....

In [ ]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str
    

In [ ]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable  \
    You are provided with a conversation between a User and an Agent. Your task....is to decide whether the Agent's \
    latest response is acceptable quality. The Agent is playing the role of {name} and is representing {name} in their \
    resume and website and so forth. The Agent has been instructed to be professional and engaging, as if talking to a \
    potential employer who came across their resume for a job. The Agent has been provided with context on {name} \
    in the form of their resume and the product requirements of a big project involving AI they worked on it their\
    present job. Here's the information: "

    evaluator_system_prompt += f"\n\n## Summary: \n{sumary}\n\n## Resume profile: \n{linkedin}\n\n"
    evaluator_system_prompt += f" With this context, please evaluate the latest repsonse, "

In [ ]:
def evaluator_user_prompt(reply, message, history):
    user_prompt =f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying whether it is acceptable answer and your feedback on what is should be."
    return user_prompt

In [ ]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [ ]:
### This is a function to evaluate what the model said....

def evaluate(reply, message, history):
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply,message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages,response_format=Evaluation)
    return response.choices[0].message.parsed

In [ ]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "Do you have experience in bioinformatics??"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [ ]:
reply

In [ ]:
evaluate(reply, "Do you have any experience in bioinformatics??", messages[:1])

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected you JACKASS!\n You jus to reply, but the quality control rejected your reply idiot!\n "
    updated_system_prompt += f"## Your attempted answer, like an idiot... \n{reply}\n\n"
    updated_system_prompt += f"## The reason for your rejection, other than you being a 'putz' is....\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    return response.choices[0].message.content

    






In [ ]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in 'pig latin'. \
            it is mandatory that you respond only and entirely in pig latin"

    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    ## Using the evaluation function again here....
    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("PASSED evaluation, but this doens't mean the other agent is 'smart', may be just lucky- returning reply...")
    else:
        print("FAILED evaluation, because you are a dumbass. Now, retrying...")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [ ]:
### Launching chat interface here...

gr.ChatInterface(chat, type="messages").launch()